<a href="https://colab.research.google.com/github/TitusChoi/Novelist_Classification/blob/titus/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 살펴보기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import random

In [ ]:
#경로 설정
import os
os.chdir('./')

In [ ]:
#파일 불러오기
train = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/new_train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/MyDrive/Novelist_Classification/datasets/sample_submission.csv', encoding = 'utf-8')

In [ ]:
#train 데이터 살펴보기
train

In [ ]:
#test 데이터 살펴보기
test

In [ ]:
#sample_submission
sample_submission

#소설가 데이터 추가

In [ ]:

# 새로운 소설데이터 폴더 주소
path = '/content/drive/MyDrive/Novelist_Classification/datasets/'
# 소설가 이름 폴더
novelist = ['Charles Dickens/','Jane Austen/','Mark Twain/']
# 소설가별 소설 txt파일 명
cd_novel = ['A Tale of Two Cities.txt','David Copperfield.txt','Great Expectations.txt','Oliver Twist.txt']
ja_novel = ['emma.txt','Persuasion.txt','Pride and Prejudice.txt','Sense and Sensibility.txt']
mt_novel = ['Adventures of Huckleberry Finn.txt','Life on the Mississippi.txt','The Innocents Abroad.txt','The Prince and the Pauper.txt']

# 데이터를 모을 리스트
novel_data = list()

# 기존 train.csv 데이터
train = pd.read_csv(path+'train.csv',encoding='utf-8')

# 형식은 기존 train.csv와 같이 index,text,author 분류

# train데이터에서 text와 author을 가져와 새로운 데이터 리스트에 저장
for i in range(len(train)):
  novel_data.append(',"'+train['text'][i]+'",'+str(train['author'][i])+'\n')

# 새로운 소설 데이터 읽기 및 novel_data에 저장
for i,name in enumerate(novelist):
  now_novel = list()
  if i == 0: now_novel = cd_novel
  elif i == 1: now_novel = ja_novel
  elif i == 2: now_novel = mt_novel
  for novel in now_novel:
    with open(path + name + novel,'r') as f:
      while True:
        line = f.readline()
        if not line:
          break
        if len(line) < 30:
          continue
        novel_data.append(',"'+line[:-1]+'",'+str(i+5)+'\n')

# 모든 데이터 셔플
random.shuffle(novel_data)

# new_train.csv에 데이터 쓰기
with open('new_train.csv','w') as f:
  f.write('index,text,author\n')
  for i, data in enumerate(novel_data):
    f.write(str(i)+data)

# **전처리**

In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [ ]:
#부호가 사라진 것을 확인할 수 있습니다.
train['text'][4]

'Have mercy gentlemen odin flung up his hands Dont write that anyway have some shame Here Ive torn my heart asunder before you and you seize the opportunity and are fingering the wounds in both halves Oh my God'

In [ ]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in base_stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
base_stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
len(base_stopwords)

153

In [ ]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
# nltk 라이브러리를 사용한 불용어 제거
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

nltk_stopword = set(stopwords.words('english'))

# Stopword 만 제거한 결과
def remove_stopwords_nltk(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in nltk_stopword:
            final_text.append(i.strip())
    return " ".join(final_text)


train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords_nltk)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords_nltk)

X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(nltk_stopword)

179

In [ ]:
train

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing janes last let...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands dont write an...,3
...,...,...,...
54874,54874,mr smith odin whispered hardly dared hope come,2
54875,54875,told plan captain us settled details accomplis...,4
54876,54876,sincere wellwisher friend sister lucy odin,1
54877,54877,wanted lend money,3


# **모델링**

In [ ]:
# FastText 사용
import gensim
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
pip install -U gensim

     |████████████████████████████████| 23.9MB 165kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
FastText = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Novelist_Classification/embbeding/fasttext.vec')

In [ ]:
Word2Vec_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Novelist_Classification/embbeding/GoogleNews-vectors-negative300.bin.gz', binary = True)

In [ ]:
#파라미터 설정
vocab_size = 47136
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
# 사전 학습된 glove 불러오고 임베딩 층에 적용시키기 
embedding_dict= dict()
f = open('/content/drive/MyDrive/Novelist_Classification/embbeding/glove.txt', encoding='utf8')

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close

embedding_matrix = np.zeros((vocab_size, 100))

for word, i in word_index.items():
    temp = embedding_dict.get(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [ ]:
# Fasttext 임베딩 과정
FT_embedding_matrix = np.zeros((vocab_size,100))

def get_vector(word):
    if word in FastText:
        return FastText[word]
    else:
        return None

for word, idx in word_index.items():
    temp = get_vector(word)
    if temp is not None:
        FT_embedding_matrix[idx] = temp
'''
for idx, word in word_index.items():
    if word in FastText:
      embedding_vector = FastText[word]
      FT_embedding_matrix[idx] = embedding_vector
'''

In [ ]:
# Word2Vec 임베딩 과정
W2V_embedding_matrix = np.zeros((vocab_size,300))

def get_vector(word):
    if word in Word2Vec_model:
        return Word2Vec_model[word]
    else:
        return None

for word,idx in word_index.items():
    temp = get_vector(word)
    if temp is not None:
        W2V_embedding_matrix[idx] = temp

In [ ]:
# Glove 임베딩 과정
vocab = nltk.FreqDist(np.hstack(train['text']))

glove = dict()
f = open('/content/drive/MyDrive/Novelist_Classification/embbeding/glove.txt',encoding='UTF8')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    glove[word] = vector

f.close()

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

# 가벼운 NLP 모델

In [ ]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 16)           754176    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 125       
Total params: 754,709
Trainable params: 754,709
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 17s - loss: 1.5650 - accuracy: 0.2754 - val_loss: 1.5452 - val_accuracy: 0.2695
Epoch 2/20
1372/1372 - 16s - loss: 1.4464 - accuracy: 0.3755 - val_loss: 1.3431 - val_accuracy: 0.3936
Epoch 3/20
1372/1372 - 15s - loss: 1.2297 - accuracy: 0.4932 - val_loss: 1.1785 - val_accuracy: 0.5155
Epoch 4/20
1372/1372 - 15s - loss: 1.1085 - accuracy: 0.5422 - val_loss: 1.1064 - val_accuracy: 0.5402
Epoch 5/20
1372/1372 - 15s - loss: 1.0281 - accuracy: 0.5877 - val_loss: 1.0500 - val_accuracy: 0.5712
Epoch 6/20
1372/1372 - 15s - loss: 0.9586 - accuracy: 0.6246 - val_loss: 1.0058 - val_accuracy: 0.6074
Epoch 7/20
1372/1372 - 15s - loss: 0.8961 - accuracy: 0.6558 - val_loss: 0.9543 - val_accuracy: 0.6311
Epoch 8/20
1372/1372 - 15s - loss: 0.8346 - accuracy: 0.6846 - val_loss: 0.9184 - val_accuracy: 0.6503
Epoch 9/20
1372/1372 - 15s - loss: 0.7783 - accuracy: 0.7087 - val_loss: 0.8838 - val_accuracy: 0.6663
Epoch 10/20
1372/1372 - 16s - loss: 0.7310 - accuracy: 0.7298 - val_loss:

In [ ]:
#가벼운 NLP모델 생성(Using FastText)
model_using_FT = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100, weights = [FT_embedding_matrix] ,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model_using_FT.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model_using_FT.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          4713600   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                2424      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 125       
Total params: 4,716,149
Trainable params: 4,716,149
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=4, verbose=1)

# fit model
num_epochs = 20
history = model_using_FT.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
1372/1372 - 80s - loss: 1.5443 - accuracy: 0.3047 - val_loss: 1.4692 - val_accuracy: 0.4145
Epoch 2/20
1372/1372 - 78s - loss: 1.3030 - accuracy: 0.4614 - val_loss: 1.1872 - val_accuracy: 0.5059
Epoch 3/20
1372/1372 - 81s - loss: 1.0778 - accuracy: 0.5659 - val_loss: 1.0403 - val_accuracy: 0.5986
Epoch 4/20
1372/1372 - 80s - loss: 0.9152 - accuracy: 0.6514 - val_loss: 0.9070 - val_accuracy: 0.6514
Epoch 5/20
1372/1372 - 78s - loss: 0.7893 - accuracy: 0.7107 - val_loss: 0.8221 - val_accuracy: 0.7005
Epoch 6/20
1372/1372 - 78s - loss: 0.6962 - accuracy: 0.7496 - val_loss: 0.7973 - val_accuracy: 0.7026
Epoch 7/20
1372/1372 - 80s - loss: 0.6245 - accuracy: 0.7775 - val_loss: 0.7405 - val_accuracy: 0.7319
Epoch 8/20
1372/1372 - 80s - loss: 0.5763 - accuracy: 0.7914 - val_loss: 0.7196 - val_accuracy: 0.7351
Epoch 9/20
1372/1372 - 80s - loss: 0.5339 - accuracy: 0.8084 - val_loss: 0.7065 - val_accuracy: 0.7458
Epoch 10/20
1372/1372 - 79s - loss: 0.5032 - accuracy: 0.8181 - val_loss:

In [ ]:
#가벼운 NLP모델 생성(Using Word2Vec)
model_using_W2V = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 300, weights = [W2V_embedding_matrix] ,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model_using_W2V.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model_using_W2V.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          14140800  
_________________________________________________________________
global_average_pooling1d_2 ( (None, 300)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                7224      
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 125       
Total params: 14,148,149
Trainable params: 14,148,149
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=4, verbose=1)

# fit model
num_epochs = 20
history = model_using_W2V.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2, callbacks = [early_stopping])

Epoch 1/20
1372/1372 - 274s - loss: 1.4404 - accuracy: 0.3739 - val_loss: 1.1875 - val_accuracy: 0.5454
Epoch 2/20
1372/1372 - 272s - loss: 1.0449 - accuracy: 0.5868 - val_loss: 0.9771 - val_accuracy: 0.6449
Epoch 3/20
1372/1372 - 255s - loss: 0.8325 - accuracy: 0.6896 - val_loss: 0.8175 - val_accuracy: 0.6887
Epoch 4/20
1372/1372 - 258s - loss: 0.6901 - accuracy: 0.7499 - val_loss: 0.7454 - val_accuracy: 0.7265
Epoch 5/20
1372/1372 - 256s - loss: 0.6018 - accuracy: 0.7821 - val_loss: 0.7372 - val_accuracy: 0.7239
Epoch 6/20
1372/1372 - 255s - loss: 0.5423 - accuracy: 0.8024 - val_loss: 0.7250 - val_accuracy: 0.7337
Epoch 7/20
1372/1372 - 248s - loss: 0.4957 - accuracy: 0.8193 - val_loss: 0.7388 - val_accuracy: 0.7259
Epoch 8/20
1372/1372 - 245s - loss: 0.4594 - accuracy: 0.8309 - val_loss: 0.7434 - val_accuracy: 0.7302
Epoch 9/20
1372/1372 - 238s - loss: 0.4287 - accuracy: 0.8422 - val_loss: 0.7582 - val_accuracy: 0.7268
Epoch 10/20
1372/1372 - 256s - loss: 0.4028 - accuracy: 0.8531 -

In [ ]:
# predict values
pred = model_using_FT.predict_proba(test_padded)

In [ ]:
pred

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,0.244451,0.130629,0.211542,0.27031,0.143069
1,1,0.244451,0.130629,0.211542,0.27031,0.143069
2,2,0.244451,0.130629,0.211542,0.27031,0.143069
3,3,0.244451,0.130629,0.211542,0.27031,0.143069
4,4,0.244451,0.130629,0.211542,0.27031,0.143069
...,...,...,...,...,...,...
19612,19612,0.244451,0.130629,0.211542,0.27031,0.143069
19613,19613,0.244451,0.130629,0.211542,0.27031,0.143069
19614,19614,0.244451,0.130629,0.211542,0.27031,0.143069
19615,19615,0.244451,0.130629,0.211542,0.27031,0.143069


In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/Novelist_Classification/datasets/submission.csv', index = False, encoding = 'utf-8')

#Result of Train  
## Base stopwords + Keras Embedding  
1372/1372 - 7s - loss: 0.4276 - accuracy: 0.8455 - val_loss: 0.7625 - val_accuracy: 0.7305
## Base stopwords + FastText  
1372/1372 - 78s - loss: 0.4719 - accuracy: 0.8304 - val_loss: 0.7002 - val_accuracy: 0.7514
## Base stopwords + Word2Vec
1372/1372 - 256s - loss: 0.4028 - accuracy: 0.8531 - val_loss: 0.7596 - val_accuracy: 0.7366





# GRU
- 참고 논문 1 : [조경현 교수님 논문](https://arxiv.org/pdf/1406.1078.pdf)
- 참고 논문 2 : [최근 논문, 변수 설정](https://github.com/AFAgarap/gru-svm/blob/master/gru_softmax_main.py)

In [ ]:
# 단방향 GRU
from keras.optimizers import Adam
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, 300, weights = [W2V_embedding_matrix] ,input_length=max_length),
  tf.keras.layers.GRU(units=256, activation='relu'), # relu를 활용한 GRU
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(2, activation='softmax'),
])

model.compile(optimizer = Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 양방향 GRU
model = Sequential([
    Embedding(vocab_size, 300, weights = [W2V_embedding_matrix] ,input_length = max_length),
    Bidirectional(GRU(256, return_sequences=True)),
    Bidirectional(GRU(256)),
    tf.keras.layers.Dropout(0.5),
    Dense(2, activation='softmax')
])
    
model.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate=0.00001))
model.summary()